In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import sys
from datetime import timedelta  

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))    
    
print("Current Working directory " , os.getcwd())

Current Working directory  C:\Users\User\GitHub\WebET_Analysis


In [2]:
data_et = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et.csv')
data_et_fixation = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et_fixation.csv')
data_et_choice = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et_choice.csv')

data_trial = pd.read_csv(
    r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial.csv')
data_trial_fixation = pd.read_csv(
    r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial_fixation.csv')
data_trial_choice = pd.read_csv(
    r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial_choice.csv')

data_subject = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_subject.csv') \
    .rename(columns=
            {
                'chosenAmount': 'bonus_pay',
                'chosenDelay': 'bonus_delay'
            }
           )    

data_prolific = pd.read_csv(r'C:/Users/User/Google Drive/Masterarbeit/Acquise/prolific_documents/prolific_export.csv') \
    .rename(columns=
            {
                'participant_id': 'prolificID',
                'age': 'age_2'
            }
           )

overview = pd.DataFrame(
    [
        [len(data_et)],
        [len(data_et_fixation)],
        [len(data_et_choice)],
        [len(data_trial)],
        [len(data_trial_fixation)],
        [len(data_trial_choice)],
        [len(data_subject)],
        [len(data_prolific)]
    ], 
    columns=['length'],
    index=['data_et', 'data_et_fixation', 'data_et_choice', 
           'data_trial', 'data_trial_fixation', 'data_trial_choice', 
           'data_subject', 'data_prolific'])
print(overview)

C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (40,56,57,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (40,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                     length
data_et              260930
data_et_fixation      78501
data_et_choice        35125
data_trial            10367
data_trial_fixation    1464
data_trial_choice      1503
data_subject             20
data_prolific            36


# Merge data_prolific into data_subject

In [3]:
data_subject = data_subject.merge(data_prolific, 
                                 on='prolificID', 
                                 how='left')
data_subject.columns

Index(['run_id', 'subject', 'chinFirst', 'choiceTask_amountLeftFirst',
       'webcam_label', 'webcam_fps', 'webcam_height', 'webcam_width',
       'user_agent', 'device', 'browser', 'browser_version', 'platform',
       'platform_version', 'bonus_pay', 'bonus_delay', 'prolificID', 'age',
       'gender', 'ethnic', 'sight', 'glasses', 'degree', 'eyeshadow',
       'masquara', 'eyeliner', 'browliner', 'vertPosition', 'triedChin',
       'keptHead', 'optionalNote', 'window_width_max', 'window_height_max',
       'window_diagonal_max', 'date', 'trial_duration_exact', 't_startTrial',
       'window_width', 'window_height', 'fps', 'choseLL', 'offset',
       'precision', 'x_mean_fixTask', 'y_mean_fixTask', 'session_id', 'status',
       'started_datetime', 'completed_date_time', 'time_taken', 'age_2',
       'num_approvals', 'num_rejections', 'prolific_score',
       'reviewed_at_datetime', 'entered_code', 'Country of Birth',
       'Current Country of Residence', 'Employment Status', 'Firs

In [12]:
data_subject['bonus_pay']

0     NaN
1     3.5
2     NaN
3     5.0
4     4.5
5     3.0
6     4.5
7     2.5
8     3.0
9     4.5
10    0.5
11    3.0
12    4.5
13    4.5
14    5.0
15    4.0
16    4.5
17    5.0
18    4.0
19    NaN
Name: bonus_pay, dtype: float64

# Finished experiment? 

In [4]:
if 'trial_length' in data_subject.columns: 
    print('trial_length already added!')
else:
    output = []
    for subject in data_trial['run_id'].unique():
        prolificID = data_trial.loc[data_trial['run_id']==subject, 'prolificID'].unique()[0]
        trial_length = len(data_trial.loc[data_trial['run_id']==subject, 'trial_index'].unique())
        output.append([subject, prolificID, trial_length])
    output = pd.DataFrame(output,
                 columns=['run_id', 'prolificID', 'trial_length']
                )

    data_subject = data_subject.merge(output, on=['run_id', 'prolificID'], how='left')

In [5]:
output = []
for subject in data_et['run_id'].unique():
    m_x_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        (data_et['task_nr']!=3), 
        'x'
    ].mean()
    m_x_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'x'
    ].mean()

    m_y_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        (data_et['task_nr']!=3), 
        'y'
    ].mean()
    m_y_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'y'
    ].mean()

    m_count_fix = data_trial.loc[
        (data_trial['run_id']==subject) &
        (data_trial['trial_type']=='eyetracking-fix-object') &
        (data_trial['task_nr']!=3), 
        'count'
    ].mean()

    m_count_choice = data_trial.loc[
        (data_trial['run_id']==subject) &
        (data_trial['trial_type']=='eyetracking-choice'), 
        'count'
    ].mean()
    
    m_choseTop = data_trial_choice.loc[data_trial_choice['run_id']==subject,
                                       'choseTop'].mean()

    output.append([subject, m_x_fix, m_y_fix, m_x_choice, m_y_choice, 
                   m_count_fix, m_count_choice, 
                   m_choseTop])

output = pd.DataFrame(output, 
                      columns=['run_id', 'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 
                               'm_count_fix', 'm_count_choice',
                               'm_choseTop'])
output

data_subject = data_subject.merge(output, on='run_id', how='left')

In [6]:
data_subject.loc[data_subject['status']=='AWAITING REVIEW', 
                 ['run_id', 'prolificID', 'status', 'trial_length',
                 'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 'm_count_fix', 'm_count_choice',
                 'm_choseTop']
                ]

,run_id,prolificID,status,trial_length,m_x_fix,m_y_fix,m_x_choice,m_y_choice,m_count_fix,m_count_choice,m_choseTop


In [7]:
if 'choice_rt' in data_subject.columns:
    print('choice_rt already added!')
else: 
    grouped = data_trial.loc[data_trial['trial_type']=='eyetracking-choice', :] \
        .groupby(['prolificID'])['trial_duration_exact'].mean() \
        .reset_index() \
        .rename(columns={'trial_duration_exact': 'choice_rt'})
    data_subject = data_subject.merge(grouped, on='prolificID', how='left')

In [8]:
data_subject.loc[:, ['run_id', 'prolificID', 'webcam_fps', 'fps', 'choice_rt']]

,run_id,prolificID,webcam_fps,fps,choice_rt
0,1,5fccc8ac636416a4288a9f3d,30.000000,2.030567,3812.6625
1,11,5c5684ef9d244c0001b29f1e,30.000000,21.456080,1659.6250
2,12,5b8969006651ea000118e42e,30.000000,17.997210,1909.5875
3,13,5b4e65de09f5af0001080f45,30.000000,21.704953,2041.9750
4,19,5f860a189bc695131f22ec9c,30.000000,21.565793,2022.9000
5,24,5f77a902fc647a327b77004a,30.000000,15.268646,2682.6250
6,25,5cf0ccf488abf10019fb775d,30.000000,13.329251,3321.0375
7,28,5f561a95aa1c4ea13672f138,30.000000,24.590346,1334.3750
8,30,5eeaa0ffaa6af11cf32ce057,29.970030,14.284995,1649.6875
9,32,5ea94df3a879f002f3e9a812,30.000000,19.953132,1186.8125


# Basic payment

In [9]:
data_pay = data_subject.loc[data_subject['status']=='APPROVED', 
                            [
                                'run_id', 'prolificID', 
                                'age',  'Country of Birth', 'Current Country of Residence', 'First Language',
                                'Nationality', 'Sex',
                                'status', 'reviewed_at_datetime', 'Country of Birth', 'entered_code',
                                'session_id', 'started_datetime', 'completed_date_time', 'time_taken',
                                'bonus_pay', 'bonus_delay'
                            ]
                           ]

In [10]:
data_pay['basic_pay'] = 2.25

# Bonus payment

In [13]:
# For two subjects, the bonus payment was not recorded
data_pay.loc[
    data_pay['prolificID'].isin(
        ['5fea6632bf9ae4a79153efdf', '5b8969006651ea000118e42e']
    ), ['bonus_pay', 'bonus_delay']] = [5, 1]

data_pay.loc[data_pay[['bonus_pay', 'bonus_delay']].isnull().any(axis=1), :]

,run_id,prolificID,age,Country of Birth,Current Country of Residence,First Language,Nationality,Sex,status,reviewed_at_datetime,Country of Birth,entered_code,session_id,started_datetime,completed_date_time,time_taken,bonus_pay,bonus_delay,basic_pay


In [14]:
data_pay['completed_date']=data_pay.apply(
    lambda x: datetime.datetime.strptime(
        x['completed_date_time'], '%Y-%m-%d %H:%M:%S.%f') \
            .date(),
    axis=1)

data_pay['reward_pound'] = data_pay['bonus_pay'] * 0.75

data_pay['due_on'] = data_pay['completed_date'] +  data_pay['bonus_delay'].map(datetime.timedelta) 

data_pay.loc[:, 
               [
                   'prolificID', 'run_id',
                   'Nationality', 'Sex',  
                   'completed_date', 'bonus_pay', 'bonus_delay', 
                   'reward_pound', 'due_on'
               ]
              ]. sort_values(by='due_on')

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\arrays\datetimelike.py:1268: PerformanceWarning: Adding/subtracting array of DateOffsets to TimedeltaArray not vectorized
  PerformanceWarning,


,prolificID,run_id,Nationality,Sex,completed_date,bonus_pay,bonus_delay,reward_pound,due_on
1,5c5684ef9d244c0001b29f1e,11,United States,Male,2021-01-18,3.5,0.0,2.625,2021-01-18
11,5f0cb319d29147695796a208,37,Italy,Male,2021-01-18,3.0,0.0,2.250,2021-01-18
8,5eeaa0ffaa6af11cf32ce057,30,Poland,Male,2021-01-18,3.0,0.0,2.250,2021-01-18
7,5f561a95aa1c4ea13672f138,28,Italy,Male,2021-01-18,2.5,0.0,1.875,2021-01-18
10,5f8472685956c40c720d0936,36,Italy,Male,2021-01-18,0.5,0.0,0.375,2021-01-18
5,5f77a902fc647a327b77004a,24,United Kingdom,Female,2021-01-18,3.0,0.0,2.250,2021-01-18
2,5b8969006651ea000118e42e,12,Italy,Female,2021-01-18,5.0,1.0,3.750,2021-01-19
14,5fcbb3066829ff1a601fe080,42,United Kingdom,Female,2021-01-18,5.0,1.0,3.750,2021-01-19
19,5fea6632bf9ae4a79153efdf,9,Serbia,Male,2021-01-18,5.0,1.0,3.750,2021-01-19
4,5f860a189bc695131f22ec9c,19,Poland,Male,2021-01-18,4.5,7.0,3.375,2021-01-25


In [15]:
bonus_due_today = data_pay.loc[data_pay['due_on']==datetime.datetime.now().date(), ['prolificID', 'reward_pound']]
bonus_due_today['reward_pound'] = bonus_due_today['reward_pound'].round(2)
bonus_due_today

,prolificID,reward_pound
2,5b8969006651ea000118e42e,3.75
14,5fcbb3066829ff1a601fe080,3.75
19,5fea6632bf9ae4a79153efdf,3.75


In [16]:
bonus_due_today.to_csv('C:/Users/User/Google Drive/Masterarbeit/Acquise/bonusToday.csv', index=False, header=False)

# Total costs

In [20]:
data_pay['basic_cost'] = data_pay['basic_pay'] * 1.4
data_pay['total_cost'] = (data_pay['basic_pay'] + data_pay['bonus_pay']) * 1.4
print('Basic Costs: ' + str(sum(data_pay['basic_cost'])))
print('Total Costs: ' + str(sum(data_pay['total_cost'])))
data_pay

Basic Costs: 59.84999999999998
Total Costs: 165.54999999999998


,run_id,prolificID,age,Country of Birth,Current Country of Residence,First Language,Nationality,Sex,status,reviewed_at_datetime,...,completed_date_time,time_taken,bonus_pay,bonus_delay,basic_pay,completed_date,reward_pound,due_on,total_cost,basic_cost
1,11,5c5684ef9d244c0001b29f1e,1991.0,Hong Kong,United States,Chinese,United States,Male,APPROVED,2021-01-18 10:53:28.738000,...,2021-01-18 08:53:34.578000,1227.532,3.5,0.0,2.25,2021-01-18,2.625,2021-01-18,8.05,3.15
2,12,5b8969006651ea000118e42e,1999.0,Italy,Italy,Italian,Italy,Female,APPROVED,2021-01-18 10:42:25.881000,...,2021-01-18 08:56:36.517000,1403.632,5.0,1.0,2.25,2021-01-18,3.750,2021-01-19,10.15,3.15
3,13,5b4e65de09f5af0001080f45,2001.0,Poland,Poland,Polish,Poland,Male,APPROVED,2021-01-18 10:56:07.011000,...,2021-01-18 08:57:22.141000,1692.194,5.0,30.0,2.25,2021-01-18,3.750,2021-02-17,10.15,3.15
4,19,5f860a189bc695131f22ec9c,2000.0,Poland,Poland,DATA EXPIRED,Poland,Male,APPROVED,2021-01-18 10:57:59.489000,...,2021-01-18 09:02:09.783000,2116.522,4.5,7.0,2.25,2021-01-18,3.375,2021-01-25,9.45,3.15
5,24,5f77a902fc647a327b77004a,1993.0,United Kingdom,United Kingdom,DATA EXPIRED,United Kingdom,Female,APPROVED,2021-01-18 17:54:59.440000,...,2021-01-18 11:35:07.643000,2115.673,3.0,0.0,2.25,2021-01-18,2.250,2021-01-18,7.35,3.15
6,25,5cf0ccf488abf10019fb775d,2001.0,New Zealand,United Kingdom,Twi,United Kingdom,Female,APPROVED,2021-01-18 17:55:09.667000,...,2021-01-18 11:31:47.394000,1913.891,4.5,90.0,2.25,2021-01-18,3.375,2021-04-18,9.45,3.15
7,28,5f561a95aa1c4ea13672f138,1983.0,Italy,United Kingdom,DATA EXPIRED,Italy,Male,APPROVED,2021-01-18 17:55:18.379000,...,2021-01-18 11:19:56.498000,1197.589,2.5,0.0,2.25,2021-01-18,1.875,2021-01-18,6.65,3.15
8,30,5eeaa0ffaa6af11cf32ce057,2000.0,Poland,Poland,Polish,Poland,Male,APPROVED,2021-01-18 17:55:26.984000,...,2021-01-18 11:22:28.884000,1351.104,3.0,0.0,2.25,2021-01-18,2.250,2021-01-18,7.35,3.15
9,32,5ea94df3a879f002f3e9a812,1997.0,Poland,Poland,Polish,Poland,Female,APPROVED,2021-01-18 17:55:32.282000,...,2021-01-18 12:13:24.296000,4402.360,4.5,90.0,2.25,2021-01-18,3.375,2021-04-18,9.45,3.15
10,36,5f8472685956c40c720d0936,1999.0,Italy,Italy,DATA EXPIRED,Italy,Male,APPROVED,2021-01-18 17:55:37.264000,...,2021-01-18 11:25:29.598000,1453.841,0.5,0.0,2.25,2021-01-18,0.375,2021-01-18,3.85,3.15


# Export modified prolific data

In [18]:
data_pay.to_csv('C:/Users/User/Google Drive/Masterarbeit/Acquise/data_pay.csv', index=False, header=True)